In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchlensmaker as tlm


class Optics(tlm.Module):
    def __init__(self):
        super().__init__()
        self.shape = tlm.Parabola(width=15., a=nn.Parameter(torch.tensor(-0.005)))

        self.optics = nn.Sequential(
            tlm.ParallelBeamUniform(width=15.),
            tlm.GapY(10.),
            
            tlm.RefractiveSurface(self.shape, (1.0, 1.49)),
            tlm.GapY(5.),
            tlm.RefractiveSurface(self.shape, (1.49, 1.0), scale=-1),
            
            tlm.GapY(80.0),
            tlm.FocalPointLoss(),
        )

    def forward(self, inputs):
        return self.optics(inputs)

optics = Optics()

tlm.render_plt(optics, (10, torch.tensor([0., 0.])))

tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-3),
    inputs = (10, torch.tensor([0., 0.])),
    num_iter = 150
)

tlm.render_plt(optics, (10, torch.tensor([0., 0.])))

In [ ]:
lens = tlm.Lens(optics.stack[2], optics.stack[3], optics.stack[4])

part = tlm.lens_to_part(lens)
part